In [31]:
import pandas_datareader.data as web
import datetime
from decimal import Decimal
from dateutil import relativedelta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, resample_apply
import pandas as pd
import talib as ta
import importlib
import os
import platform
import psutil
import numpy as np

#https://www.sbisec.co.jp/ETGate/WPLETmgR001Control?OutSide=on&getFlg=on&burl=search_domestic&cat1=domestic&cat2=none&dir=info&file=domestic_info150818_01.html
#https://www.morningstar.co.jp/etf_foreign/search.jsp
# start = datetime.date(2021,1,1)
start = datetime.datetime.today() + relativedelta.relativedelta(years=-5)
end = datetime.date.today()
# data = web.DataReader('7203.T', 'yahoo', start, end)  #toyota
# data = web.DataReader('1321.T', 'yahoo', start, end)  #(NEXT FUNDS)日経225連動型上場投信
# data = web.DataReader('1570.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均レバレッジ・インデックス連動型上場投信
# data = web.DataReader('1357.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均ダブルインバース・インデックス連動型上場投信
# data = web.DataReader('1655.T', 'yahoo', start, end)  #iシェアーズ S&P500 米国株 ETF　→　取得されるデータがおかしい？？
# data = web.DataReader('2558.T', 'yahoo', start, end)  #MAXIS 米国株式(S&P500)上場投信
# data = web.DataReader('2631.T', 'yahoo', start, end)  #MAXIS ナスダック100上場投信
data = web.DataReader('2559.T', 'yahoo', start, end)    #MAXIS 全世界株式(オール・カントリー)上場投信

# data = web.DataReader('BTC-JPY', 'yahoo', start, end)   #Bitcoin
data = data.astype("double")
print(data)

initial = 1000_000
fname = f"html/plot-"

#position
#0 ... display "only long" result
#1 ... display both "only long" and "long and short" result
position = 1

df_returns = pd.DataFrame()

               High      Low     Open    Close   Volume  Adj Close
Date                                                              
2020-01-07   9926.0   9926.0   9926.0   9926.0      0.0     9926.0
2020-01-08   9926.0   9926.0   9926.0   9926.0      0.0     9926.0
2020-01-09  10080.0  10080.0  10080.0  10080.0      0.0    10080.0
2020-01-10  10310.0  10280.0  10290.0  10310.0   1285.0    10310.0
2020-01-14  10400.0  10350.0  10390.0  10370.0   4223.0    10370.0
...             ...      ...      ...      ...      ...        ...
2022-09-13  14845.0  14800.0  14845.0  14820.0   3573.0    14820.0
2022-09-14  14515.0  14375.0  14430.0  14390.0  12573.0    14390.0
2022-09-15  14410.0  14325.0  14405.0  14400.0   4748.0    14400.0
2022-09-16  14215.0  14145.0  14190.0  14200.0   8385.0    14200.0
2022-09-20  14310.0  14215.0  14310.0  14240.0   4940.0    14240.0

[661 rows x 6 columns]


**Methods**

In [14]:
def execute_backtest(bt):
    output = bt.run()
    print(output)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html") # 実行結果（グラフ）
    df_temp = pd.DataFrame(output).transpose().drop(["_equity_curve", "_trades"], axis=1)
    global df_returns
    df_returns = pd.concat([df_returns, df_temp], ignore_index=True)

In [15]:
def execute_optimize(bt, opt_params):
    output = bt.optimize(**opt_params)
    print(output)
    display(output._trades)
    display(output._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html")
    df_temp = pd.DataFrame(output).transpose().drop(["_equity_curve", "_trades"], axis=1)
    df_temp["_strategy"] = str(output._strategy) + "_optimize"
    global df_returns
    df_returns = pd.concat([df_returns, df_temp], ignore_index=True)

**MACD CROSS**

In [51]:
import BacktestingStrategies.Strategy_MacdCross as btstMACD
importlib.reload(btstMACD)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstMACD.MACDCross, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True, #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)

#最適化
opt_params = {
    "MACDshort": range(10, 25, 5),\
    "MACDlong": range(10, 75, 10),\
    "MACDsignal": range(10, 15, 5),\
    "maximize": 'Equity Final [$]',
    "method": 'grid',\
    "constraint": lambda x: (x.MACDlong > x.MACDshort) & (x.MACDshort > x.MACDsignal),\
    "max_tries": 1000
}
execute_optimize(bt, opt_params)

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstMACD.MACDCross_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True, #自動でポジションをクローズ(オープン)
    )
    execute_backtest(bt)

    #最適化
    execute_optimize(bt, opt_params)

Start                     2021-09-21 00:00:00
End                       2022-09-16 00:00:00
Duration                    360 days 00:00:00
Exposure Time [%]                   42.213115
Equity Final [$]                 945555.74475
Equity Peak [$]                  1027741.0215
Return [%]                          -5.444426
Buy & Hold Return [%]                 9.06298
Return (Ann.) [%]                   -5.617822
Volatility (Ann.) [%]               10.723715
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -10.905025
Avg. Drawdown [%]                   -6.123963
Max. Drawdown Duration      170 days 00:00:00
Avg. Drawdown Duration       87 days 00:00:00
# Trades                                    7
Win Rate [%]                        28.571429
Best Trade [%]                       8.109186
Worst Trade [%]                     -4.883089
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,69,67,78,14340.63650,14005.0,-23158.91850,-0.023405,2021-12-28,2022-01-17,20 days
1,70,91,103,13777.86450,13200.0,-40450.51500,-0.041942,2022-02-03,2022-02-22,19 days
2,70,118,137,13194.99350,14265.0,74900.45500,0.081092,2022-03-16,2022-04-13,28 days
3,69,144,145,14606.94825,14200.0,-28079.42925,-0.027860,2022-04-22,2022-04-25,3 days
4,70,167,178,13953.73075,13400.0,-38761.15250,-0.039683,2022-05-31,2022-06-15,15 days
5,69,188,227,13637.17150,14350.0,49185.16650,0.052271,2022-06-29,2022-08-25,57 days
6,66,240,243,14918.48275,14190.0,-48079.86150,-0.048831,2022-09-13,2022-09-16,3 days


Start                     2021-09-21 00:00:00
End                       2022-09-16 00:00:00
Duration                    360 days 00:00:00
Exposure Time [%]                   35.245902
Equity Final [$]                    942575.74
Equity Peak [$]                   1069200.669
Return [%]                          -5.742426
Buy & Hold Return [%]                 9.06298
Return (Ann.) [%]                   -5.925013
Volatility (Ann.) [%]                9.987523
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -11.842953
Avg. Drawdown [%]                   -3.590715
Max. Drawdown Duration      148 days 00:00:00
Avg. Drawdown Duration       42 days 00:00:00
# Trades                                    6
Win Rate [%]                             50.0
Best Trade [%]                       3.741014
Worst Trade [%]                     -5.098737
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,73,97,99,13667.32000,13720.0,3845.6400,0.003854,2022-02-14,2022-02-16,2 days
1,73,101,102,13566.82500,13310.0,-18748.2250,-0.018930,2022-02-18,2022-02-21,3 days
2,72,120,148,13581.89925,14090.0,36583.2540,0.037410,2022-03-18,2022-04-28,41 days
3,71,170,179,14220.04250,13495.0,-51478.0175,-0.050987,2022-06-03,2022-06-16,13 days
4,70,195,232,13767.81500,14060.0,20452.9500,0.021222,2022-07-08,2022-09-01,55 days
5,66,240,243,14918.48275,14190.0,-48079.8615,-0.048831,2022-09-13,2022-09-16,3 days


<Strategy MACDCross(MACDshort=20,MACDlong=60,MACDsignal=10)>

Start                     2021-09-21 00:00:00
End                       2022-09-16 00:00:00
Duration                    360 days 00:00:00
Exposure Time [%]                   42.213115
Equity Final [$]                 945555.74475
Equity Peak [$]                  1027741.0215
Return [%]                          -5.444426
Buy & Hold Return [%]                 9.06298
Return (Ann.) [%]                   -5.617822
Volatility (Ann.) [%]               10.723715
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -10.905025
Avg. Drawdown [%]                   -6.123963
Max. Drawdown Duration      170 days 00:00:00
Avg. Drawdown Duration       87 days 00:00:00
# Trades                                    7
Win Rate [%]                        28.571429
Best Trade [%]                       8.109186
Worst Trade [%]                     -4.883089
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,69,67,78,14340.63650,14005.0,-23158.91850,-0.023405,2021-12-28,2022-01-17,20 days
1,70,91,103,13777.86450,13200.0,-40450.51500,-0.041942,2022-02-03,2022-02-22,19 days
2,70,118,137,13194.99350,14265.0,74900.45500,0.081092,2022-03-16,2022-04-13,28 days
3,69,144,145,14606.94825,14200.0,-28079.42925,-0.027860,2022-04-22,2022-04-25,3 days
4,70,167,178,13953.73075,13400.0,-38761.15250,-0.039683,2022-05-31,2022-06-15,15 days
5,69,188,227,13637.17150,14350.0,49185.16650,0.052271,2022-06-29,2022-08-25,57 days
6,66,240,243,14918.48275,14190.0,-48079.86150,-0.048831,2022-09-13,2022-09-16,3 days


Start                     2021-09-21 00:00:00
End                       2022-09-16 00:00:00
Duration                    360 days 00:00:00
Exposure Time [%]                   43.852459
Equity Final [$]                  1010079.254
Equity Peak [$]                 1143942.91425
Return [%]                           1.007925
Buy & Hold Return [%]                 9.06298
Return (Ann.) [%]                    1.041144
Volatility (Ann.) [%]               12.468945
Sharpe Ratio                         0.083499
Sortino Ratio                        0.115063
Calmar Ratio                         0.088972
Max. Drawdown [%]                  -11.701953
Avg. Drawdown [%]                   -5.087724
Max. Drawdown Duration      148 days 00:00:00
Avg. Drawdown Duration       56 days 00:00:00
# Trades                                    6
Win Rate [%]                        66.666667
Best Trade [%]                       5.447861
Worst Trade [%]                     -4.883089
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-71,79,103,13960.55150,13200.0,53999.15650,0.054479,2022-01-18,2022-02-22,35 days
1,77,120,147,13581.89925,13755.0,13328.75775,0.012745,2022-03-18,2022-04-27,40 days
2,76,169,179,13968.80500,13495.0,-36009.18000,-0.033919,2022-06-02,2022-06-16,14 days
3,76,193,194,13466.33000,13510.0,3318.92000,0.003243,2022-07-06,2022-07-07,1 days
4,75,195,231,13767.81500,14130.0,27163.87500,0.026307,2022-07-08,2022-08-31,54 days
5,71,240,243,14918.48275,14190.0,-51722.27525,-0.048831,2022-09-13,2022-09-16,3 days


<Strategy MACDCross_WithShortPosition(MACDshort=15,MACDlong=60,MACDsignal=10)>

**EMA CROSS**

In [52]:
import BacktestingStrategies.Strategy_EMACross as btstEMA
importlib.reload(btstEMA)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstEMA.EmaCrossStrategy, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)

#最適化
opt_params = {
    "EMAshort": range(10, 100, 10),\
    "EMAlong": range(10, 200, 5),\
    "maximize": 'Equity Final [$]',\
    "method": 'grid',\
    "constraint":  lambda x: (x.EMAlong > x.EMAshort),\
    "max_tries":  1000
}
execute_optimize(bt, opt_params)

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstEMA.EmaCrossStrategy_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True) #自動でポジションをクローズ(オープン)
    execute_backtest(bt)

    #最適化
    execute_optimize(bt, opt_params)

Start                     2021-09-21 00:00:00
End                       2022-09-16 00:00:00
Duration                    360 days 00:00:00
Exposure Time [%]                   61.885246
Equity Final [$]                 954330.74525
Equity Peak [$]                 1086343.07525
Return [%]                          -4.566925
Buy & Hold Return [%]                 9.06298
Return (Ann.) [%]                   -4.713076
Volatility (Ann.) [%]               13.315554
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -14.212834
Avg. Drawdown [%]                   -3.253937
Max. Drawdown Duration      148 days 00:00:00
Avg. Drawdown Duration       42 days 00:00:00
# Trades                                   11
Win Rate [%]                        36.363636
Best Trade [%]                       4.071313
Worst Trade [%]                     -5.043316
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,74,14,47,13365.83500,13910.0,40268.21000,0.040713,2021-10-12,2021-11-30,49 days
1,74,55,59,14034.12675,14005.0,-2155.37950,-0.002075,2021-12-10,2021-12-16,6 days
2,73,66,78,14129.59700,14005.0,-9095.58100,-0.008818,2021-12-27,2022-01-17,21 days
3,74,91,99,13777.86450,13720.0,-4281.97300,-0.004200,2022-02-03,2022-02-16,13 days
4,75,101,102,13566.82500,13310.0,-19261.87500,-0.018930,2022-02-18,2022-02-21,3 days
5,73,119,147,13642.19625,13755.0,8234.67375,0.008269,2022-03-17,2022-04-27,41 days
6,72,167,177,13953.73075,13250.0,-50668.61400,-0.050433,2022-05-31,2022-06-14,14 days
7,70,188,191,13637.17150,13390.0,-17302.00500,-0.018125,2022-06-29,2022-07-04,5 days
8,70,193,194,13466.33000,13510.0,3056.90000,0.003243,2022-07-06,2022-07-07,1 days
9,68,195,230,13767.81500,14265.0,33808.58000,0.036112,2022-07-08,2022-08-30,53 days


Start                     2021-09-21 00:00:00
End                       2022-09-16 00:00:00
Duration                    360 days 00:00:00
Exposure Time [%]                   15.983607
Equity Final [$]                  1023885.244
Equity Peak [$]                   1069245.244
Return [%]                           2.388524
Buy & Hold Return [%]                 9.06298
Return (Ann.) [%]                    2.467795
Volatility (Ann.) [%]                6.750593
Sharpe Ratio                         0.365567
Sortino Ratio                        0.492284
Calmar Ratio                         0.559774
Max. Drawdown [%]                    -4.40856
Avg. Drawdown [%]                   -1.974453
Max. Drawdown Duration       24 days 00:00:00
Avg. Drawdown Duration        8 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                       2.393803
Worst Trade [%]                      2.393803
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,72,205,243,13858.2605,14190.0,23885.244,0.023938,2022-07-25,2022-09-16,53 days


<Strategy EmaCrossStrategy(EMAshort=10,EMAlong=175)>

Start                     2021-09-21 00:00:00
End                       2022-09-16 00:00:00
Duration                    360 days 00:00:00
Exposure Time [%]                   61.885246
Equity Final [$]                 954330.74525
Equity Peak [$]                 1086343.07525
Return [%]                          -4.566925
Buy & Hold Return [%]                 9.06298
Return (Ann.) [%]                   -4.713076
Volatility (Ann.) [%]               13.315554
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -14.212834
Avg. Drawdown [%]                   -3.253937
Max. Drawdown Duration      148 days 00:00:00
Avg. Drawdown Duration       42 days 00:00:00
# Trades                                   11
Win Rate [%]                        36.363636
Best Trade [%]                       4.071313
Worst Trade [%]                     -5.043316
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,74,14,47,13365.83500,13910.0,40268.21000,0.040713,2021-10-12,2021-11-30,49 days
1,74,55,59,14034.12675,14005.0,-2155.37950,-0.002075,2021-12-10,2021-12-16,6 days
2,73,66,78,14129.59700,14005.0,-9095.58100,-0.008818,2021-12-27,2022-01-17,21 days
3,74,91,99,13777.86450,13720.0,-4281.97300,-0.004200,2022-02-03,2022-02-16,13 days
4,75,101,102,13566.82500,13310.0,-19261.87500,-0.018930,2022-02-18,2022-02-21,3 days
5,73,119,147,13642.19625,13755.0,8234.67375,0.008269,2022-03-17,2022-04-27,41 days
6,72,167,177,13953.73075,13250.0,-50668.61400,-0.050433,2022-05-31,2022-06-14,14 days
7,70,188,191,13637.17150,13390.0,-17302.00500,-0.018125,2022-06-29,2022-07-04,5 days
8,70,193,194,13466.33000,13510.0,3056.90000,0.003243,2022-07-06,2022-07-07,1 days
9,68,195,230,13767.81500,14265.0,33808.58000,0.036112,2022-07-08,2022-08-30,53 days


Start                     2021-09-21 00:00:00
End                       2022-09-16 00:00:00
Duration                    360 days 00:00:00
Exposure Time [%]                   22.131148
Equity Final [$]                 1033347.1815
Equity Peak [$]                  1078707.1815
Return [%]                           3.334718
Buy & Hold Return [%]                 9.06298
Return (Ann.) [%]                    3.445916
Volatility (Ann.) [%]                9.657624
Sharpe Ratio                         0.356808
Sortino Ratio                        0.521898
Calmar Ratio                         0.420914
Max. Drawdown [%]                   -8.186752
Avg. Drawdown [%]                   -3.399807
Max. Drawdown Duration       24 days 00:00:00
Avg. Drawdown Duration       11 days 00:00:00
# Trades                                    2
Win Rate [%]                            100.0
Best Trade [%]                       2.481343
Worst Trade [%]                      0.857896
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-74,162,180,13428.19975,13095.0,24656.7815,0.024813,2022-05-24,2022-06-17,24 days
1,72,209,243,14069.30000,14190.0,8690.4000,0.008579,2022-07-29,2022-09-16,49 days


<Strategy EmaCrossStrategy_WithShortPosition(EMAshort=20,EMAlong=145)>

**SMA**

In [53]:
import BacktestingStrategies.Strategy_SMACross as btstSMA
importlib.reload(btstSMA)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstSMA.SmaCross, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)

# #最適化
opt_params = {
    "SMA_short": range(10, 100, 5),\
    "SMA_long": range(10, 200, 5),\
    "maximize": 'Equity Final [$]',\
    "method": 'grid',\
    "max_tries": 1000,\
    "constraint": lambda x: x.SMA_short < x.SMA_long}
execute_optimize(bt, opt_params)

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstSMA.SmaCross_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )
    execute_backtest(bt)

    # #最適化
    execute_optimize(bt, opt_params)


Start                     2021-09-21 00:00:00
End                       2022-09-16 00:00:00
Duration                    360 days 00:00:00
Exposure Time [%]                   37.704918
Equity Final [$]                  827288.8705
Equity Peak [$]                     1000000.0
Return [%]                         -17.271113
Buy & Hold Return [%]                 9.06298
Return (Ann.) [%]                  -17.783797
Volatility (Ann.) [%]                8.596564
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -20.510828
Avg. Drawdown [%]                  -20.510828
Max. Drawdown Duration      255 days 00:00:00
Avg. Drawdown Duration      255 days 00:00:00
# Trades                                    5
Win Rate [%]                             20.0
Best Trade [%]                       3.456428
Worst Trade [%]                     -7.876262
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,69,71,83,14491.37900,13350.0,-78755.1510,-0.078763,2022-01-05,2022-01-24,19 days
1,64,124,152,14310.48800,13670.0,-40991.2320,-0.044757,2022-03-25,2022-05-10,46 days
2,60,174,183,14471.28000,13420.0,-63076.8000,-0.072646,2022-06-09,2022-06-22,13 days
3,58,202,238,14054.22575,14540.0,28174.9065,0.034564,2022-07-20,2022-09-09,51 days
4,58,241,243,14501.42850,14190.0,-18062.8530,-0.021476,2022-09-14,2022-09-16,2 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 513 configurations.
  output = _optimize_grid()


Start                     2021-09-21 00:00:00
End                       2022-09-16 00:00:00
Duration                    360 days 00:00:00
Exposure Time [%]                   17.622951
Equity Final [$]                1103888.48125
Equity Peak [$]                 1105013.48125
Return [%]                          10.388848
Buy & Hold Return [%]                 9.06298
Return (Ann.) [%]                   10.747157
Volatility (Ann.) [%]                6.955555
Sharpe Ratio                         1.545119
Sortino Ratio                        2.983058
Calmar Ratio                         3.378885
Max. Drawdown [%]                   -3.180682
Avg. Drawdown [%]                    -1.32377
Max. Drawdown Duration       30 days 00:00:00
Avg. Drawdown Duration       13 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                      10.525826
Worst Trade [%]                     10.525826
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,75,180,222,13159.82025,14545.0,103888.48125,0.105258,2022-06-17,2022-08-18,62 days


...

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-73,50,83,13577.45725,13350.0,16604.37925,0.016753,2021-12-03,2022-01-24,52 days
1,71,124,152,14310.48800,13670.0,-45474.64800,-0.044757,2022-03-25,2022-05-10,46 days
2,67,174,183,14471.28000,13420.0,-70435.76000,-0.072646,2022-06-09,2022-06-22,13 days
3,64,202,238,14054.22575,14540.0,31089.55200,0.034564,2022-07-20,2022-09-09,51 days
4,64,241,243,14501.42850,14190.0,-19931.42400,-0.021476,2022-09-14,2022-09-16,2 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 513 configurations.
  output = _optimize_grid()


Start                     2021-09-21 00:00:00
End                       2022-09-16 00:00:00
Duration                    360 days 00:00:00
Exposure Time [%]                   21.311475
Equity Final [$]                1094289.51225
Equity Peak [$]                 1142799.51225
Return [%]                           9.428951
Buy & Hold Return [%]                 9.06298
Return (Ann.) [%]                    9.752712
Volatility (Ann.) [%]                8.389481
Sharpe Ratio                         1.162493
Sortino Ratio                        1.918723
Calmar Ratio                         2.297546
Max. Drawdown [%]                   -4.244839
Avg. Drawdown [%]                   -1.902779
Max. Drawdown Duration       14 days 00:00:00
Avg. Drawdown Duration        8 days 00:00:00
# Trades                                    2
Win Rate [%]                            100.0
Best Trade [%]                       9.082037
Worst Trade [%]                      0.427493
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,75,180,219,13159.82025,14355.0,89638.48125,0.090820,2022-06-17,2022-08-15,59 days
1,77,232,243,14129.59700,14190.0,4651.03100,0.004275,2022-09-01,2022-09-16,15 days


<Strategy SmaCross_WithShortPosition(SMA_short=75,SMA_long=135)>

**ボリンジャーバンド**

In [54]:
import BacktestingStrategies.Strategy_BBD as btstBBD
importlib.reload(btstBBD)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstBBD.BBsigma, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)

# #最適化
opt_params = {
    "n": range(10, 40, 5),\
    "nu": np.arange(2.0, 3.1, 0.1).round(2).tolist(),\
    "nd": np.arange(2.0, 3.1, 0.1).round(2).tolist(),\
    "maximize": 'Equity Final [$]',\
    "method": 'grid',\
    "max_tries": 1000}
execute_optimize(bt, opt_params)

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstBBD.BBsigma_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )
    execute_backtest(bt)

    # #最適化
    execute_optimize(bt, opt_params)

Start                     2021-09-21 00:00:00
End                       2022-09-16 00:00:00
Duration                    360 days 00:00:00
Exposure Time [%]                    38.52459
Equity Final [$]                  1062938.148
Equity Peak [$]                  1086339.0605
Return [%]                           6.293815
Buy & Hold Return [%]                 9.06298
Return (Ann.) [%]                    6.506744
Volatility (Ann.) [%]               14.631279
Sharpe Ratio                         0.444715
Sortino Ratio                        0.670429
Calmar Ratio                         0.663779
Max. Drawdown [%]                   -9.802579
Avg. Drawdown [%]                   -4.018207
Max. Drawdown Duration      177 days 00:00:00
Avg. Drawdown Duration       39 days 00:00:00
# Trades                                    3
Win Rate [%]                        66.666667
Best Trade [%]                       4.128757
Worst Trade [%]                     -0.986921
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,71,47,71,13978.8545,14420.0,31321.3305,0.031558,2021-11-30,2022-01-05,36 days
1,76,83,123,13416.0825,13970.0,42097.7300,0.041288,2022-01-24,2022-03-24,59 days
2,75,148,175,14159.7455,14020.0,-10480.9125,-0.009869,2022-04-28,2022-06-10,43 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 726 configurations.
  output = _optimize_grid()


Start                     2021-09-21 00:00:00
End                       2022-09-16 00:00:00
Duration                    360 days 00:00:00
Exposure Time [%]                   44.672131
Equity Final [$]                   1196093.75
Equity Peak [$]                    1196093.75
Return [%]                          19.609375
Buy & Hold Return [%]                 9.06298
Return (Ann.) [%]                   20.313649
Volatility (Ann.) [%]               16.838421
Sharpe Ratio                         1.206387
Sortino Ratio                         2.15818
Calmar Ratio                         2.062619
Max. Drawdown [%]                   -9.848474
Avg. Drawdown [%]                   -3.337964
Max. Drawdown Duration       68 days 00:00:00
Avg. Drawdown Duration       18 days 00:00:00
# Trades                                    2
Win Rate [%]                            100.0
Best Trade [%]                      10.417017
Worst Trade [%]                      8.481883
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,75,85,127,13245.2410,14625.0,103481.925,0.104170,2022-01-26,2022-03-30,63 days
1,82,155,220,13315.5875,14445.0,92611.825,0.084819,2022-05-13,2022-08-16,95 days


...

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,71,47,71,13978.85450,14420.0,31321.33050,0.031558,2021-11-30,2022-01-05,36 days
1,-73,72,123,14124.73475,13970.0,11295.63675,0.010955,2022-01-06,2022-03-24,77 days
2,-73,125,126,14179.46250,14430.0,-18289.23750,-0.017669,2022-03-28,2022-03-29,1 days
3,-70,127,128,14552.60625,14450.0,7182.43750,0.007051,2022-03-30,2022-03-31,1 days
4,72,148,175,14159.74550,14020.0,-10061.67600,-0.009869,2022-04-28,2022-06-10,43 days
5,-73,203,204,13900.84850,13975.0,-5413.05950,-0.005334,2022-07-21,2022-07-22,1 days
6,-71,216,220,14149.61100,14445.0,-20972.61900,-0.020876,2022-08-09,2022-08-16,7 days
7,-68,221,222,14428.22500,14545.0,-7940.70000,-0.008094,2022-08-17,2022-08-18,1 days
8,-68,241,243,14358.57150,14190.0,11462.86200,0.011740,2022-09-14,2022-09-16,2 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 726 configurations.
  output = _optimize_grid()


Start                     2021-09-21 00:00:00
End                       2022-09-16 00:00:00
Duration                    360 days 00:00:00
Exposure Time [%]                   53.278689
Equity Final [$]                1185322.51975
Equity Peak [$]                 1223797.51975
Return [%]                          18.532252
Buy & Hold Return [%]                 9.06298
Return (Ann.) [%]                   19.194826
Volatility (Ann.) [%]               17.803498
Sharpe Ratio                         1.078149
Sortino Ratio                        1.940176
Calmar Ratio                         1.949015
Max. Drawdown [%]                   -9.848474
Avg. Drawdown [%]                   -2.611628
Max. Drawdown Duration       68 days 00:00:00
Avg. Drawdown Duration       16 days 00:00:00
# Trades                                    3
Win Rate [%]                            100.0
Best Trade [%]                       9.095788
Worst Trade [%]                       0.29712
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,75,85,128,13245.24100,14450.0,90356.92500,0.090958,2022-01-26,2022-03-31,64 days
1,81,155,220,13315.58750,14445.0,91482.41250,0.084819,2022-05-13,2022-08-16,95 days
2,-81,222,241,14473.00225,14430.0,3483.18225,0.002971,2022-08-18,2022-09-14,27 days


<Strategy BBsigma_WithShortPosition(n=35,nu=2.2,nd=2.5)>

**RSI**

In [55]:
import BacktestingStrategies.Strategy_RsiOscillator as btstRsi
importlib.reload(btstRsi)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstRsi.RsiOscillator, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)

# #最適化
opt_params = {
    "upper_bound": range(50, 85, 5),
    "lower_bound": range(10, 45, 5),
    "rsi_window": range(10, 30, 2),\
    "maximize": 'Equity Final [$]',\
    "method": 'grid',\
    "max_tries": 1000}
execute_optimize(bt, opt_params)

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstRsi.RsiOscillator_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )
    execute_backtest(bt)

    # #最適化
    execute_optimize(bt, opt_params)

Start                     2021-09-21 00:00:00
End                       2022-09-16 00:00:00
Duration                    360 days 00:00:00
Exposure Time [%]                   55.327869
Equity Final [$]                 1077218.7595
Equity Peak [$]                  1092388.7595
Return [%]                           7.721876
Buy & Hold Return [%]                 9.06298
Return (Ann.) [%]                    7.984905
Volatility (Ann.) [%]               16.514586
Sharpe Ratio                         0.483506
Sortino Ratio                        0.733466
Calmar Ratio                         0.714383
Max. Drawdown [%]                  -11.177339
Avg. Drawdown [%]                   -4.960447
Max. Drawdown Duration      148 days 00:00:00
Avg. Drawdown Duration       44 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                       7.728745
Worst Trade [%]                      7.728745
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,74,88,222,13501.50325,14545.0,77218.7595,0.077287,2022-01-31,2022-08-18,199 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 490 configurations.
  output = _optimize_grid()


Start                     2021-09-21 00:00:00
End                       2022-09-16 00:00:00
Duration                    360 days 00:00:00
Exposure Time [%]                   39.344262
Equity Final [$]                1219188.87425
Equity Peak [$]                 1219188.87425
Return [%]                          21.918887
Buy & Hold Return [%]                 9.06298
Return (Ann.) [%]                   22.713683
Volatility (Ann.) [%]               15.992576
Sharpe Ratio                         1.420264
Sortino Ratio                        2.652229
Calmar Ratio                         2.298797
Max. Drawdown [%]                   -9.880682
Avg. Drawdown [%]                   -2.090663
Max. Drawdown Duration       41 days 00:00:00
Avg. Drawdown Duration       11 days 00:00:00
# Trades                                    4
Win Rate [%]                            100.0
Best Trade [%]                       5.710595
Worst Trade [%]                      4.099519
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,73,49,71,13647.22100,14420.0,56412.86700,0.056625,2021-12-02,2022-01-05,34 days
1,78,88,122,13501.50325,14055.0,43172.74650,0.040995,2022-01-31,2022-03-23,51 days
2,80,157,174,13622.09725,14400.0,62232.22000,0.057106,2022-05-17,2022-06-09,23 days
3,87,184,203,13310.56275,13970.0,57371.04075,0.049542,2022-06-23,2022-07-21,28 days


...

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-73,17,71,13552.58100,14420.0,-63321.5870,-0.064004,2021-10-15,2022-01-05,82 days
1,69,83,123,13416.08250,13970.0,38220.3075,0.041288,2022-01-24,2022-03-24,59 days
2,-68,135,138,14283.94275,14400.0,-7891.8930,-0.008125,2022-04-11,2022-04-14,3 days
3,-66,142,174,14647.13600,14400.0,16310.9760,0.016873,2022-04-20,2022-06-09,50 days
4,72,179,203,13561.80025,13970.0,29390.3820,0.030099,2022-06-16,2022-07-21,35 days
5,-72,214,219,13930.70000,14355.0,-30549.6000,-0.030458,2022-08-05,2022-08-15,10 days
6,-70,229,239,14020.25450,14670.0,-45482.1850,-0.046343,2022-08-29,2022-09-12,14 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 490 configurations.
  output = _optimize_grid()


Start                     2021-09-21 00:00:00
End                       2022-09-16 00:00:00
Duration                    360 days 00:00:00
Exposure Time [%]                   44.672131
Equity Final [$]                1219208.88125
Equity Peak [$]                 1224790.66725
Return [%]                          21.920888
Buy & Hold Return [%]                 9.06298
Return (Ann.) [%]                   22.715763
Volatility (Ann.) [%]               16.705402
Sharpe Ratio                         1.359785
Sortino Ratio                        2.664504
Calmar Ratio                         3.075207
Max. Drawdown [%]                   -7.386743
Avg. Drawdown [%]                   -2.846752
Max. Drawdown Duration       54 days 00:00:00
Avg. Drawdown Duration       19 days 00:00:00
# Trades                                    7
Win Rate [%]                        71.428571
Best Trade [%]                       7.211732
Worst Trade [%]                      -1.88979
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-71,54,68,14005.32875,14270.0,-18791.65875,-0.018898,2021-12-09,2021-12-29,20 days
1,-69,76,97,14124.73475,13600.0,36206.69775,0.037150,2022-01-13,2022-02-14,32 days
2,77,104,122,13109.57275,14055.0,72797.89825,0.072117,2022-02-24,2022-03-23,27 days
3,81,155,171,13315.58750,14000.0,55437.41250,0.051399,2022-05-13,2022-06-06,24 days
4,86,181,203,13300.51325,13970.0,57575.86050,0.050335,2022-06-20,2022-07-21,31 days
5,-86,208,214,13920.74950,14000.0,-6815.54300,-0.005693,2022-07-28,2022-08-05,8 days
6,-82,238,243,14468.02700,14190.0,22798.21400,0.019217,2022-09-09,2022-09-16,7 days


<Strategy RsiOscillator_WithShortPosition(upper_bound=55,lower_bound=40,rsi_window=18)>

**ドンチャンブレイクアウト**

In [33]:
import BacktestingStrategies.Strategy_Donchian as btstdnchn
importlib.reload(btstdnchn)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstdnchn.DnchnBreakout, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)

# #最適化
opt_params = {
    "dnchn_long": range(10, 50, 5),
    "dnchn_short": range(10, 50, 5),
    "maximize": 'Equity Final [$]',\
    "method": 'grid',\
    "constraint": lambda x: (x.dnchn_long > x.dnchn_short),\
    "max_tries": 1000}
execute_optimize(bt, opt_params)

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstdnchn.DnchnBreakout_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )
    execute_backtest(bt)

    # #最適化
    execute_optimize(bt, opt_params)


Start                     2020-01-07 00:00:00
End                       2022-09-20 00:00:00
Duration                    987 days 00:00:00
Exposure Time [%]                   49.016641
Equity Final [$]                 953424.59825
Equity Peak [$]                  1141215.4225
Return [%]                           -4.65754
Buy & Hold Return [%]               43.461616
Return (Ann.) [%]                   -1.801893
Volatility (Ann.) [%]               11.866603
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -17.091499
Avg. Drawdown [%]                   -3.747496
Max. Drawdown Duration      522 days 00:00:00
Avg. Drawdown Duration       72 days 00:00:00
# Trades                                   16
Win Rate [%]                             50.0
Best Trade [%]                       9.576643
Worst Trade [%]                     -8.439336
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,94,31,33,10562.02450,10050.0,-48130.30300,-0.048478,2020-02-21,2020-02-26,5 days
1,112,65,107,8441.58000,9250.0,90543.04000,0.095766,2020-04-13,2020-06-16,64 days
2,107,129,166,9727.91600,10020.0,31252.98800,0.030025,2020-07-16,2020-09-10,56 days
3,104,185,194,10290.68800,10040.0,-26071.55200,-0.024361,2020-10-12,2020-10-23,11 days
4,98,210,259,10662.51950,11190.0,51693.08900,0.049471,2020-11-17,2021-01-29,73 days
5,93,265,276,11727.76650,11850.0,11367.71550,0.010423,2021-02-08,2021-02-25,17 days
6,90,288,316,12270.43950,12450.0,16160.44500,0.014634,2021-03-15,2021-04-22,38 days
7,87,324,328,12813.11250,12460.0,-30720.78750,-0.027559,2021-05-10,2021-05-14,4 days
8,84,339,355,12953.80550,12970.0,1360.33800,0.001250,2021-05-31,2021-06-22,22 days
9,81,403,414,13466.33000,13370.0,-7802.73000,-0.007153,2021-09-01,2021-09-16,15 days


Start                     2020-01-07 00:00:00
End                       2022-09-20 00:00:00
Duration                    987 days 00:00:00
Exposure Time [%]                   68.078669
Equity Final [$]                 1379899.5765
Equity Peak [$]                  1518507.5265
Return [%]                          37.989958
Buy & Hold Return [%]               43.461616
Return (Ann.) [%]                   13.061705
Volatility (Ann.) [%]               14.895161
Sharpe Ratio                         0.876909
Sortino Ratio                        1.451019
Calmar Ratio                         1.148563
Max. Drawdown [%]                  -11.372216
Avg. Drawdown [%]                   -2.238894
Max. Drawdown Duration      259 days 00:00:00
Avg. Drawdown Duration       23 days 00:00:00
# Trades                                    4
Win Rate [%]                             75.0
Best Trade [%]                      25.204929
Worst Trade [%]                      -3.59104
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,116,78,175,8572.2235,9650.0,125022.0740,0.125729,2020-05-01,2020-09-25,147 days
1,105,210,499,10662.5195,13350.0,282185.4525,0.252049,2020-11-17,2022-01-24,433 days
2,100,539,569,14039.1515,13535.0,-50415.1500,-0.035910,2022-03-24,2022-05-11,48 days
3,96,630,660,14069.3000,14310.0,23107.2000,0.017108,2022-08-05,2022-09-20,46 days


<Strategy DnchnBreakout(dnchn_long=35,dnchn_short=30)>

Start                     2020-01-07 00:00:00
End                       2022-09-20 00:00:00
Duration                    987 days 00:00:00
Exposure Time [%]                   71.104387
Equity Final [$]                 393509.23275
Equity Peak [$]                   1080439.472
Return [%]                         -60.649077
Buy & Hold Return [%]               43.461616
Return (Ann.) [%]                   -29.92221
Volatility (Ann.) [%]               14.895305
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -66.653683
Avg. Drawdown [%]                  -36.729868
Max. Drawdown Duration      922 days 00:00:00
Avg. Drawdown Duration      470 days 00:00:00
# Trades                                   27
Win Rate [%]                        40.740741
Best Trade [%]                       9.552284
Worst Trade [%]                    -30.378896
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,94,31,33,10562.02450,10050.0,-48130.30300,-0.048478,2020-02-21,2020-02-26,5 days
1,-98,34,35,9701.73750,9350.0,34470.27500,0.036255,2020-02-27,2020-02-28,1 days
2,-110,36,42,8955.45000,8100.0,94099.50000,0.095523,2020-03-02,2020-03-10,8 days
3,-149,45,46,7214.11250,7950.0,-109647.23750,-0.102007,2020-03-13,2020-03-16,3 days
4,-136,51,107,7094.70650,9250.0,-293119.91600,-0.303789,2020-03-24,2020-06-16,84 days
5,69,129,166,9727.91600,10020.0,20153.79600,0.030025,2020-07-16,2020-09-10,56 days
6,-72,175,194,9602.23250,10040.0,-31519.26000,-0.045590,2020-09-25,2020-10-23,28 days
7,-69,198,199,9602.23250,9580.0,1534.04250,0.002315,2020-10-29,2020-10-30,1 days
8,-70,200,259,9452.97500,11190.0,-121591.75000,-0.183754,2020-11-02,2021-01-29,88 days
9,46,265,276,11727.76650,11850.0,5622.74100,0.010423,2021-02-08,2021-02-25,17 days


Start                     2020-01-07 00:00:00
End                       2022-09-20 00:00:00
Duration                    987 days 00:00:00
Exposure Time [%]                   70.801815
Equity Final [$]                1617516.64175
Equity Peak [$]                  1764413.8595
Return [%]                          61.751664
Buy & Hold Return [%]               43.461616
Return (Ann.) [%]                    20.12174
Volatility (Ann.) [%]               19.692359
Sharpe Ratio                         1.021804
Sortino Ratio                        1.963478
Calmar Ratio                          1.78347
Max. Drawdown [%]                  -11.282352
Avg. Drawdown [%]                   -2.808209
Max. Drawdown Duration      286 days 00:00:00
Avg. Drawdown Duration       30 days 00:00:00
# Trades                                   18
Win Rate [%]                        61.111111
Best Trade [%]                      21.641044
Worst Trade [%]                     -4.078717
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-124,42,45,8059.90500,7250.0,100428.22000,0.100486,2020-03-10,2020-03-13,3 days
1,-139,46,51,7910.64750,7130.0,108510.00250,0.098683,2020-03-16,2020-03-24,8 days
2,138,82,166,8743.06500,10020.0,176217.03000,0.146051,2020-05-12,2020-09-10,121 days
3,-144,175,198,9602.23250,9650.0,-6878.52000,-0.004975,2020-09-25,2020-10-29,34 days
4,-144,199,200,9532.57900,9500.0,4691.37600,0.003418,2020-10-30,2020-11-02,3 days
5,129,210,355,10662.51950,12970.0,297664.98450,0.216410,2020-11-17,2021-06-22,217 days
6,-130,376,396,12925.69950,13080.0,-20059.06500,-0.011937,2021-07-21,2021-08-23,33 days
7,123,403,417,13466.33000,12960.0,-62278.59000,-0.037600,2021-09-01,2021-09-22,21 days
8,115,434,463,13838.16150,13910.0,8261.42750,0.005191,2021-10-18,2021-11-30,43 days
9,-118,464,497,13582.43250,13630.0,-5612.96500,-0.003502,2021-12-01,2022-01-20,50 days


<Strategy DnchnBreakout_WithShortPosition(dnchn_long=40,dnchn_short=15)>

In [57]:
df_returns_sorted = df_returns[["_strategy", "Start", "End", "Duration", "Exposure Time [%]", "Equity Final [$]", "Equity Peak [$]", "Return [%]", "Buy & Hold Return [%]", "Return (Ann.) [%]", "Volatility (Ann.) [%]", "Sharpe Ratio", "Sortino Ratio", "Calmar Ratio", "Max. Drawdown [%]", "Avg. Drawdown [%]", "Max. Drawdown Duration", "Avg. Drawdown Duration", "# Trades", "Win Rate [%]", "Best Trade [%]", "Worst Trade [%]", "Avg. Trade [%]", "Max. Trade Duration", "Avg. Trade Duration", "Profit Factor", "Expectancy [%]", "SQN", 
]].sort_values(by = "Equity Final [$]", ascending = False)
df_returns_sorted.to_csv("OriginalStrategies.csv")